Подключение библиотек

In [1]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

Для практики был выбран несбалансированный датасет Ecoli (определение кишечной палочки) с семью столбцами вещественного типа и без пропущенных значений.
Записи внутри файла dat были частично изменены, чтобы легче привести в DataFrame правильный "Заголовок".

In [23]:
data = pd.read_csv('ecoli1.dat', 
                 sep=",\s+",
                 index_col=None,
                 header=9)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336 entries, 0 to 335
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Mcg     336 non-null    float64
 1   Gvh     336 non-null    float64
 2   Lip     336 non-null    float64
 3   Chg     336 non-null    float64
 4   Aac     336 non-null    float64
 5   Alm1    336 non-null    float64
 6   Alm2    336 non-null    float64
 7   Class   336 non-null    object 
dtypes: float64(7), object(1)
memory usage: 21.1+ KB


<ipython-input-23-b325c9228b04>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_csv('ecoli1.dat',


In [24]:
data

,Mcg,Gvh,Lip,Chg,Aac,Alm1,Alm2,Class
0,0.49,0.29,0.48,0.5,0.56,0.24,0.35,negative
1,0.07,0.40,0.48,0.5,0.54,0.35,0.44,negative
2,0.56,0.40,0.48,0.5,0.49,0.37,0.46,negative
3,0.59,0.49,0.48,0.5,0.52,0.45,0.36,negative
4,0.23,0.32,0.48,0.5,0.55,0.25,0.35,negative
...,...,...,...,...,...,...,...,...
331,0.74,0.56,0.48,0.5,0.47,0.68,0.30,negative
332,0.71,0.57,0.48,0.5,0.48,0.35,0.32,negative
333,0.61,0.60,0.48,0.5,0.44,0.39,0.38,negative
334,0.59,0.61,0.48,0.5,0.42,0.42,0.37,negative


In [40]:
y_ds = data['Class'].to_numpy()
X_ds = data.drop('Class',1).to_numpy()
print(np.unique(y))

[0 1]


Так как в целевой переменной находятся категориальные данные, их следует преобразовать в числовые.

In [42]:
from sklearn.preprocessing import LabelEncoder
y_ds = LabelEncoder().fit_transform(y_ds)
print(np.unique(y_ds))
print(data['Class'])
print(y_ds)

[0 1]
0      negative
1      negative
2      negative
3      negative
4      negative
         ...   
331    negative
332    negative
333    negative
334    negative
335    negative
Name: Class, Length: 336, dtype: object
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0]


Так как пустых значений нет, то далее следует проверить набор на несбалансированность. Коэффициент несбалансированности данных присутсвует, но не переходит границу в 5 единиц, что хоть как-то радует.

In [39]:
counts = data.Class.value_counts()
print(counts)
print(counts[0]/counts[1])

negative    259
positive     77
Name: Class, dtype: int64
3.3636363636363638


В качестве метода балансировки данных не следует использовать методы сокращения данных (under-sampling), так как количество экземпляров не доходит и до тысячи, что может сказаться на результатах, из-за чего будут применяться методы добавления данных (over-sampling).

In [62]:
from imblearn.over_sampling import SMOTE # Методы over-sampling'а
from imblearn.over_sampling import ADASYN
from imblearn.over_sampling import BorderlineSMOTE
oversamplers = [SMOTE, ADASYN, BorderlineSMOTE]

from sklearn.model_selection import cross_validate, StratifiedKFold # Инструменты для кросс-валидации, нормализаици и оценки
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import balanced_accuracy_score

from sklearn.svm import SVC # Используемые классификаторы 
from sklearn.naive_bayes import GaussianNB
# from sklearn.naive_bayes import BernoulliNB # Не будет использоваться, так как все данные имеют непрерывный характер, что вряд ли хорошо скажется на результативности метода Бернулли
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
classifiers = [SVC, GaussianNB, LinearDiscriminantAnalysis, DecisionTreeClassifier]

from itertools import product # Инструмент для перемножения списков

В качестве быстрого проведения тестирования и просмотра результатов была создана общая функция, в которой будет меняться лишь модель (классификатор) и метод балансировки данных

In [67]:
def CheckModel(model, oversampler):
  y = y_ds.copy()
  X = X_ds.copy()
  kf = StratifiedKFold(5)
  train_accuracies = []
  test_accuracies = []
  for train_index, test_index in kf.split(X, y):
    X_train = X[train_index]
    X_test = X[test_index]
    y_train = y[train_index]
    y_test = y[test_index]
    X_train, y_train = oversampler.fit_resample(X_train, y_train)
    scaler = RobustScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    model.fit(X_train, y_train)
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    balanced_accuracy_tr = balanced_accuracy_score(y_pred_train, y_train)
    balanced_accuracy_ts = balanced_accuracy_score(y_pred_test, y_test)
    train_accuracies.append(balanced_accuracy_tr)
    test_accuracies.append(balanced_accuracy_ts)
  return np.mean(train_accuracies) * 100, np.mean(test_accuracies) * 100

In [74]:
lastclf = None
best_test = 0.0
best_oversampler = None
for clf, oversampler in product(classifiers, oversamplers):
  if lastclf != clf:
    lastclf = clf
    if best_oversampler != None:
      print(f'\n Лучше всего метод проявился себя с {best_oversampler.__name__} - {best_test} %')
    best_test = 0.0
    print(f'\n\n\tMethod - {clf.__name__}:')
  print(f'- {oversampler.__name__}')
  train_acc, test_acc = CheckModel(clf(), oversampler())
  print("Обучение:", train_acc, "%, Тест:", test_acc, ' %')
  if best_test < test_acc:
    best_test = test_acc
    best_oversampler = oversampler
print(f'\n Лучше всего метод проявился себя с {best_oversampler.__name__} - {best_test} %')



	Method - SVC:
- SMOTE
Обучение: 94.08886456153411 %, Тест: 84.47842555994728  %
- ADASYN
Обучение: 93.30095689452413 %, Тест: 83.40235690235691  %
- BorderlineSMOTE
Обучение: 93.5504659761277 %, Тест: 84.22850135380735  %

 Лучше всего метод проявился себя с SMOTE - 84.47842555994728 %


	Method - GaussianNB:
- SMOTE
Обучение: 88.54927429463511 %, Тест: 74.54934135534886  %
- ADASYN
Обучение: 86.30233458457838 %, Тест: 71.25935646351954  %
- BorderlineSMOTE
Обучение: 87.79065720411108 %, Тест: 75.53758469199646  %

 Лучше всего метод проявился себя с BorderlineSMOTE - 75.53758469199646 %


	Method - LinearDiscriminantAnalysis:
- SMOTE
Обучение: 91.31577350904712 %, Тест: 82.37850489842428  %
- ADASYN
Обучение: 90.75871622283472 %, Тест: 83.36831989645287  %
- BorderlineSMOTE
Обучение: 92.3647728022954 %, Тест: 82.41390394203692  %

 Лучше всего метод проявился себя с ADASYN - 83.36831989645287 %


	Method - DecisionTreeClassifier:
- SMOTE
Обучение: 100.0 %, Тест: 83.70438160910297  

Без настройки параметров неплохо проявили себя метод опорных векторов и дерево принятия решений, но неизвестно, что случилось бы с результатам, если бы пришлось подбирать параметры через, например, GridSearchCV.